## Step 0: Latent Dirichlet Allocation ##

LDA is used to classify text in a document to a particular topic. It builds a topic per document model and words per topic model, modeled as Dirichlet distributions. 

* Each document is modeled as a multinomial distribution of topics and each topic is modeled as a multinomial distribution of words.
* LDA assumes that the every chunk of text we feed into it will contain words that are somehow related. Therefore choosing the right corpus of data is crucial. 
* It also assumes documents are produced from a mixture of topics. Those topics then generate words based on their probability distribution. 

## Step 1: Load the dataset

The dataset we'll use is a list of over one million news headlines published over a period of 15 years. We'll start by loading it from the `abcnews-date-text.csv` file.

In [5]:
'''
Load the dataset from the CSV and save it to 'data_text'
'''
import pandas as pd
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);
# We only need the Headlines text column from the data
data_text = data[:300000][['headline_text']];
data_text['index'] = data_text.index

documents = data_text

Let's glance at the dataset:

In [6]:
'''
Get the total number of documents
'''
print(len(documents))

300000


In [7]:
documents[:5]

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


## Step 2: Data Preprocessing ##

We will perform the following steps:

* **Tokenization**: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
* Words that have fewer than 3 characters are removed.
* All **stopwords** are removed.
* Words are **lemmatized** - words in third person are changed to first person and verbs in past and future tenses are changed into present.
* Words are **stemmed** - words are reduced to their root form.


In [9]:
'''
Loading Gensim and nltk libraries
'''
# pip install gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)

D:\Users\YangY\Anaconda2\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [10]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\YangY\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Lemmatizer Example
Before preprocessing our dataset, let's first look at an lemmatizing example. What would be the output if we lemmatized the word 'went':

In [11]:
print(WordNetLemmatizer().lemmatize('went', pos = 'v')) # past tense to present tense

go


### Stemmer Example
Let's also look at a stemming example. Let's throw a number of words at the stemmer and see how it deals with each one:

In [12]:
stemmer = SnowballStemmer("english")
original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in original_words]

pd.DataFrame(data={'original word':original_words, 'stemmed':singles })

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [57]:
'''
Write a function to perform the pre processing steps on the entire dataset
'''
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
            
    return result



In [58]:
'''
Preview a document after preprocessing
'''
document_num = 4310
doc_sample = documents[documents['index'] == document_num].values[0][0]

print("Original document: ")
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print("\n\nTokenized and lemmatized document: ")
print(preprocess(doc_sample))

Original document: 
['rain', 'helps', 'dampen', 'bushfires']


Tokenized and lemmatized document: 
[u'rain', u'help', u'dampen', u'bushfir']


In [59]:
documents

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4
5,ambitious olsson wins triple jump,5
6,antic delighted with record breaking barca,6
7,aussie qualifier stosur wastes four memphis match,7
8,aust addresses un security council over iraq,8
9,australia is locked into war timetable opp,9


Let's now preprocess all the news headlines we have. To do that, let's use the [map](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.map.html) function from pandas to apply `preprocess()` to the `headline_text` column

**Note**: This may take a few minutes (it take 6 minutes on my laptop)

In [60]:
# TODO: preprocess all the headlines, saving the list of results as 'processed_docs'
processed_docs = documents['headline_text'].map(preprocess)

In [61]:
'''
Preview 'processed_docs'
'''
processed_docs[:10]

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

## Step 3.1: Bag of words on the dataset

Now let's create a dictionary from 'processed_docs' containing the number of times a word appears in the training set. To do that, let's pass `processed_docs` to [`gensim.corpora.Dictionary()`](https://radimrehurek.com/gensim/corpora/dictionary.html) and call it '`dictionary`'.

In [64]:
'''
Create a dictionary from 'processed_docs' containing the number of times a word appears 
in the training set using gensim.corpora.Dictionary and call it 'dictionary'
'''
dictionary = gensim.corpora.Dictionary(processed_docs)

In [65]:
'''
Checking dictionary created
'''
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

(7958, u'verplank')
(13916, u'woodi')
(14482, u'francesco')
(17648, u'kaniva')
(23126, u'yanagisawa')
(12187, u'scold')
(11412, u'szabic')
(6913, u'emptiv')
(23207, u'strikebreak')
(6219, u'citrus')
(23771, u'donger')


** Gensim filter_extremes **

[`filter_extremes(no_below=5, no_above=0.5, keep_n=100000)`](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.filter_extremes)

Filter out tokens that appear in

* less than no_below documents (absolute number) or
* more than no_above documents (fraction of total corpus size, not absolute number).
* after (1) and (2), keep only the first keep_n most frequent tokens (or keep all if None).

In [66]:
dictionary.filter_extremes(no_below=15, no_above=0.9)

** Gensim doc2bow **

[`doc2bow(document)`](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.doc2bow)

* Convert document (a list of words) into the bag-of-words format = list of (token_id, token_count) 2-tuples. Each word is assumed to be a tokenized and normalized string (either unicode or utf8-encoded). No further preprocessing is done on the words in document; apply tokenization, stemming etc. before calling this method.

In [67]:
'''
Create the Bag-of-words model for each document i.e for each document we create a dictionary reporting how many
words and how many times those words appear. Save this to 'bow_corpus'
'''

bow_corpus = [dictionary.doc2bow(document) for document in processed_docs]

In [68]:
'''
Checking Bag of Words corpus for our sample document --> (token_id, token_count)
'''
bow_corpus[document_num]

[(71, 1), (107, 1), (462, 1), (3530, 1)]

In [69]:
'''
Preview BOW for our sample preprocessed document
'''
# Here document_num is document number 4310 which we have checked in Step 2
bow_doc_4310 = bow_corpus[document_num]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

Word 71 ("bushfir") appears 1 time.
Word 107 ("help") appears 1 time.
Word 462 ("rain") appears 1 time.
Word 3530 ("dampen") appears 1 time.


## Step 3.2: TF-IDF on our document set ##

While performing TF-IDF on the corpus is not necessary for LDA implemention using the gensim model, it is recemmended. TF-IDF expects a bag-of-words (integer values) training corpus during initialization. During transformation, it will take a vector and return another vector of the same dimensionality.

*Please note: The author of Gensim dictates the standard procedure for LDA to be using the Bag of Words model.*

** TF-IDF stands for "Term Frequency, Inverse Document Frequency".**

* It is a way to score the importance of words (or "terms") in a document based on how frequently they appear across multiple documents.
* If a word appears frequently in a document, it's important. Give the word a high score. But if a word appears in many documents, it's not a unique identifier. Give the word a low score.
* Therefore, common words like "the" and "for", which appear in many documents, will be scaled down. Words that appear frequently in a single document will be scaled up.

In other words:

* TF(w) = `(Number of times term w appears in a document) / (Total number of terms in the document)`.
* IDF(w) = `log_e(Total number of documents / Number of documents with term w in it)`.

** For example **

* Consider a document containing `100` words wherein the word 'tiger' appears 3 times. 
* The term frequency (i.e., tf) for 'tiger' is then: 
    - `TF = (3 / 100) = 0.03`. 

* Now, assume we have `10 million` documents and the word 'tiger' appears in `1000` of these. Then, the inverse document frequency (i.e., idf) is calculated as:
    - `IDF = log(10,000,000 / 1,000) = 4`. 

* Thus, the Tf-idf weight is the product of these quantities: 
    - `TF-IDF = 0.03 * 4 = 0.12`.

In [71]:
'''
Create tf-idf model object using models.TfidfModel on 'bow_corpus' and save it to 'tfidf'
'''
from gensim import corpora, models


tfidf = models.TfidfModel(bow_corpus)

In [76]:
'''
Apply transformation to the entire corpus and call it 'corpus_tfidf'
'''

corpus_tfidf = tfidf[bow_corpus]

In [77]:
'''
Preview TF-IDF scores for our first document --> --> (token_id, tfidf score)
'''
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5959813347777092),
 (1, 0.39204529549491984),
 (2, 0.48531419274988147),
 (3, 0.50554610985785686)]


## Step 4.1: Running LDA using Bag of Words ##

We are going for 10 topics in the document corpus.

** We will be running LDA using all CPU cores to parallelize and speed up model training.**

Some of the parameters we will be tweaking are:

* **num_topics** is the number of requested latent topics to be extracted from the training corpus.
* **id2word** is a mapping from word ids (integers) to words (strings). It is used to determine the vocabulary size, as well as for debugging and topic printing.
* **workers** is the number of extra processes to use for parallelization. Uses all available cores by default.
* **alpha** and **eta** are hyperparameters that affect sparsity of the document-topic (theta) and topic-word (lambda) distributions. We will let these be the default values for now(default value is `1/num_topics`)
    - Alpha is the per document topic distribution.
        * High alpha: Every document has a mixture of all topics(documents appear similar to each other).
        * Low alpha: Every document has a mixture of very few topics

    - Eta is the per topic word distribution.
        * High eta: Each topic has a mixture of most words(topics appear similar to each other).
        * Low eta: Each topic has a mixture of few words.

* ** passes ** is the number of training passes through the corpus. For  example, if the training corpus has 50,000 documents, chunksize is  10,000, passes is 2, then online training is done in 10 updates: 
    * `#1 documents 0-9,999 `
    * `#2 documents 10,000-19,999 `
    * `#3 documents 20,000-29,999 `
    * `#4 documents 30,000-39,999 `
    * `#5 documents 40,000-49,999 `
    * `#6 documents 0-9,999 `
    * `#7 documents 10,000-19,999 `
    * `#8 documents 20,000-29,999 `
    * `#9 documents 30,000-39,999 `
    * `#10 documents 40,000-49,999` 

In [80]:
# LDA mono-core -- fallback code in case LdaMulticore throws an error on your machine
# lda_model = gensim.models.LdaModel(bow_corpus, 
#                                    num_topics = 10, 
#                                    id2word = dictionary,                                    
#                                    passes = 50)

# LDA multicore 
'''
Train your lda model using gensim.models.LdaMulticore and save it to 'lda_model'
'''

lda_model = gensim.models.LdaModel(bow_corpus,
                                   num_topics=10,
                                   id2word=dictionary,
                                   passes=2)

In [90]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Topic: 0 
Words: 0.028*"iraq" + 0.024*"open" + 0.022*"kill" + 0.019*"report" + 0.018*"howard" + 0.016*"secur" + 0.016*"troop" + 0.016*"deal" + 0.014*"deni" + 0.014*"decis"


Topic: 1 
Words: 0.021*"labor" + 0.021*"sydney" + 0.020*"reject" + 0.016*"govt" + 0.015*"support" + 0.014*"return" + 0.013*"communiti" + 0.013*"get" + 0.012*"work" + 0.012*"warn"


Topic: 2 
Words: 0.026*"closer" + 0.023*"health" + 0.021*"minist" + 0.021*"back" + 0.016*"take" + 0.016*"australia" + 0.015*"feder" + 0.013*"firefight" + 0.012*"blaze" + 0.012*"state"


Topic: 3 
Words: 0.025*"nation" + 0.017*"break" + 0.015*"land" + 0.014*"timor" + 0.013*"bomb" + 0.013*"leav" + 0.013*"park" + 0.013*"concern" + 0.013*"forc" + 0.013*"stand"


Topic: 4 
Words: 0.026*"jail" + 0.022*"rise" + 0.017*"price" + 0.017*"case" + 0.017*"high" + 0.015*"protest" + 0.015*"teen" + 0.014*"worker" + 0.013*"year" + 0.012*"farm"


Topic: 5 
Words: 0.071*"polic" + 0.030*"charg" + 0.028*"crash" + 0.026*"court" + 0.024*"face" + 0.020*"investig

## Step 4.2 Running LDA using TF-IDF ##

In [82]:
'''
Define lda model using corpus_tfidf, again using gensim.models.LdaMulticore()
'''

lda_model_tfidf = models.LdaMulticore(corpus_tfidf,
                                      num_topics=10,
                                      id2word=dictionary,
                                      passes=2)

In [92]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model_tfidf.print_topics(-1):
    print("Topic: {} \nWord: {}".format(idx, topic))
    print("\n")

Topic: 0 
Word: 0.012*"govt" + 0.010*"health" + 0.010*"fund" + 0.008*"plan" + 0.008*"urg" + 0.007*"servic" + 0.006*"boost" + 0.006*"council" + 0.006*"worker" + 0.006*"indigen"


Topic: 1 
Word: 0.012*"assault" + 0.009*"charg" + 0.009*"guilti" + 0.007*"water" + 0.007*"restrict" + 0.006*"plead" + 0.006*"face" + 0.005*"court" + 0.005*"council" + 0.005*"speed"


Topic: 2 
Word: 0.009*"rudd" + 0.008*"east" + 0.007*"power" + 0.007*"timor" + 0.006*"law" + 0.006*"station" + 0.006*"worri" + 0.006*"govt" + 0.006*"wast" + 0.005*"south"


Topic: 3 
Word: 0.009*"govt" + 0.008*"farm" + 0.008*"drought" + 0.007*"nurs" + 0.007*"wind" + 0.006*"costello" + 0.006*"plan" + 0.006*"council" + 0.006*"budget" + 0.005*"care"


Topic: 4 
Word: 0.012*"price" + 0.009*"coast" + 0.007*"north" + 0.007*"nuclear" + 0.006*"korea" + 0.006*"gold" + 0.006*"iran" + 0.006*"open" + 0.006*"titl" + 0.006*"world"


Topic: 5 
Word: 0.034*"closer" + 0.007*"murray" + 0.007*"bird" + 0.006*"break" + 0.006*"season" + 0.005*"refus" + 0

### Classification of the topics ###

As we can see, when using tf-idf, heavier weights are given to words that are not as frequent which results in nouns being factored in. That makes it harder to figure out the categories as nouns can be hard to categorize. This goes to show that the models we apply depend on the type of corpus of text we are dealing with. 

## Step 5.1: Performance evaluation by classifying sample document using LDA Bag of Words model##

We will check to see where our test document would be classified. 

In [84]:
'''
Text of sample document 4310
'''
processed_docs[4310]

[u'rain', u'help', u'dampen', u'bushfir']

In [85]:
'''
Check which topic our test document belongs to using the LDA Bag of Words model.
'''
document_num = 4310

# Our test document is document number 4310
for index, score in sorted(lda_model[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.597509503365	 
Topic: 0.025*"nation" + 0.017*"break" + 0.015*"land" + 0.014*"timor" + 0.013*"bomb" + 0.013*"leav" + 0.013*"park" + 0.013*"concern" + 0.013*"forc" + 0.013*"stand"

Score: 0.242489889264	 
Topic: 0.046*"govt" + 0.037*"urg" + 0.024*"fund" + 0.019*"council" + 0.019*"help" + 0.017*"boost" + 0.017*"push" + 0.017*"plan" + 0.015*"hospit" + 0.015*"opposit"

Score: 0.0200005527586	 
Topic: 0.062*"water" + 0.018*"elect" + 0.016*"die" + 0.015*"talk" + 0.014*"coast" + 0.014*"centr" + 0.014*"market" + 0.014*"plan" + 0.013*"rudd" + 0.013*"lead"

Score: 0.019999999553	 
Topic: 0.028*"iraq" + 0.024*"open" + 0.022*"kill" + 0.019*"report" + 0.018*"howard" + 0.016*"secur" + 0.016*"troop" + 0.016*"deal" + 0.014*"deni" + 0.014*"decis"

Score: 0.019999999553	 
Topic: 0.021*"labor" + 0.021*"sydney" + 0.020*"reject" + 0.016*"govt" + 0.015*"support" + 0.014*"return" + 0.013*"communiti" + 0.013*"get" + 0.012*"work" + 0.012*"warn"

Score: 0.019999999553	 
Topic: 0.026*"closer" + 0.023*"h

### It has the highest probability (`x`) to be  part of the topic that we assigned as X, which is the accurate classification. ###

## Step 5.2: Performance evaluation by classifying sample document using LDA TF-IDF model##

In [86]:
'''
Check which topic our test document belongs to using the LDA TF-IDF model.
'''
# Our test document is document number 4310
for index, score in sorted(lda_model_tfidf[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.432348966599	 
Topic: 0.010*"court" + 0.010*"murder" + 0.008*"hick" + 0.007*"liber" + 0.007*"polic" + 0.007*"trial" + 0.007*"case" + 0.006*"accus" + 0.006*"charg" + 0.006*"face"

Score: 0.407623589039	 
Topic: 0.009*"rudd" + 0.008*"east" + 0.007*"power" + 0.007*"timor" + 0.006*"law" + 0.006*"station" + 0.006*"worri" + 0.006*"govt" + 0.006*"wast" + 0.005*"south"

Score: 0.0200041644275	 
Topic: 0.012*"govt" + 0.010*"health" + 0.010*"fund" + 0.008*"plan" + 0.008*"urg" + 0.007*"servic" + 0.006*"boost" + 0.006*"council" + 0.006*"worker" + 0.006*"indigen"

Score: 0.0200041420758	 
Topic: 0.010*"blaze" + 0.010*"firefight" + 0.007*"crew" + 0.007*"polic" + 0.007*"drink" + 0.007*"climat" + 0.006*"solomon" + 0.006*"raid" + 0.005*"drug" + 0.005*"uranium"

Score: 0.0200038123876	 
Topic: 0.009*"govt" + 0.008*"farm" + 0.008*"drought" + 0.007*"nurs" + 0.007*"wind" + 0.006*"costello" + 0.006*"plan" + 0.006*"council" + 0.006*"budget" + 0.005*"care"

Score: 0.0200037769973	 
Topic: 0.022*"cra

### It has the highest probability (`x%`) to be  part of the topic that we assigned as X. ###

## Step 6: Testing model on unseen document ##

In [95]:
unseen_document = "My favorite sports activities are running and swimming."

# Data preprocessing step for the unseen document
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}\t Word: {}".format(score, index, lda_model.print_topic(index, 5)))

Score: 0.22000002861	 Topic: 1	 Word: 0.021*"labor" + 0.021*"sydney" + 0.020*"reject" + 0.016*"govt" + 0.015*"support"
Score: 0.22000002861	 Topic: 7	 Word: 0.028*"aust" + 0.023*"school" + 0.020*"green" + 0.020*"want" + 0.018*"australian"
Score: 0.22000002861	 Topic: 9	 Word: 0.020*"world" + 0.019*"record" + 0.018*"industri" + 0.017*"win" + 0.015*"time"
Score: 0.219999909401	 Topic: 3	 Word: 0.025*"nation" + 0.017*"break" + 0.015*"land" + 0.014*"timor" + 0.013*"bomb"
Score: 0.019999999553	 Topic: 0	 Word: 0.028*"iraq" + 0.024*"open" + 0.022*"kill" + 0.019*"report" + 0.018*"howard"
Score: 0.019999999553	 Topic: 2	 Word: 0.026*"closer" + 0.023*"health" + 0.021*"minist" + 0.021*"back" + 0.016*"take"
Score: 0.019999999553	 Topic: 4	 Word: 0.026*"jail" + 0.022*"rise" + 0.017*"price" + 0.017*"case" + 0.017*"high"
Score: 0.019999999553	 Topic: 5	 Word: 0.071*"polic" + 0.030*"charg" + 0.028*"crash" + 0.026*"court" + 0.024*"face"
Score: 0.019999999553	 Topic: 6	 Word: 0.046*"govt" + 0.037*"urg"

The model correctly classifies the unseen document with 'x'% probability to the X category.